In [1]:
import cv2
import os
import numpy as np
from os.path import isfile, join
from moviepy.editor import VideoFileClip
import csv
import mediapipe as mp 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay

# Transformer les données en panda

In [2]:
#transformer csv en pandas
df = pd.read_csv('/Users/paulinetournier/Desktop/main4/codeprojet2/main4_capgemini/pre-traitement/output/coords.csv')
df.head()

,class,x1_1,y1_1,z1_1,v1_1,x2_1,y2_1,z2_1,v2_1,x3_1,...,z73_20,v73_20,x74_20,y74_20,z74_20,v74_20,x75_20,y75_20,z75_20,v75_20
0,adresse,0.534457,0.262017,-0.749250,0.999847,0.548606,0.217810,-0.694320,0.999842,0.558305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adresse,0.536288,0.246847,-0.734284,0.999638,0.553682,0.206800,-0.683484,0.999495,0.564451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adresse,0.536795,0.229008,-0.702487,0.999740,0.556249,0.191573,-0.654676,0.999570,0.569983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adresse,0.533269,0.242183,-0.815752,0.999703,0.552650,0.199815,-0.748720,0.999527,0.566371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adresse,0.537291,0.246702,-0.757443,0.999768,0.555579,0.205676,-0.713148,0.999679,0.569113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#remplacer nan par 0
df = df.fillna(0)
df

,class,x1_1,y1_1,z1_1,v1_1,x2_1,y2_1,z2_1,v2_1,x3_1,...,z73_20,v73_20,x74_20,y74_20,z74_20,v74_20,x75_20,y75_20,z75_20,v75_20
0,adresse,0.534457,0.262017,-0.749250,0.999847,0.548606,0.217810,-0.694320,0.999842,0.558305,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,adresse,0.536288,0.246847,-0.734284,0.999638,0.553682,0.206800,-0.683484,0.999495,0.564451,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,adresse,0.536795,0.229008,-0.702487,0.999740,0.556249,0.191573,-0.654676,0.999570,0.569983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,adresse,0.533269,0.242183,-0.815752,0.999703,0.552650,0.199815,-0.748720,0.999527,0.566371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,adresse,0.537291,0.246702,-0.757443,0.999768,0.555579,0.205676,-0.713148,0.999679,0.569113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,adresse,0.536716,0.230147,-0.771241,0.999735,0.557405,0.189963,-0.719911,0.999565,0.570912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,adresse,0.540579,0.231800,-0.857348,0.999727,0.561003,0.191351,-0.805514,0.999567,0.574979,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,adresse,0.544509,0.235925,-0.980898,0.999740,0.564778,0.195267,-0.908327,0.999587,0.578075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,adresse,0.534644,0.248341,-0.821114,0.999484,0.552726,0.204001,-0.763463,0.999271,0.566524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,adresse,0.537579,0.241163,-0.796832,0.999747,0.558902,0.196899,-0.730430,0.999593,0.573524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Séparation des données (entrainement, test)

In [4]:
#séparation des données en données de test et d'entrainement
y=df[['class']]
X=df.iloc[:,1:6001]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)


# KNN plus proche voisin

In [5]:
#test du classifieur KNN plus proche voisins
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=2)

In [6]:
y_predict=model.predict(X_test)
print(y_predict)
print(y_test)

['adresse' 'affaire' 'adresse' 'affaire' 'affaire' 'affaire' 'affaire'
 'adresse']
      class
6   adresse
30  affaire
9   adresse
26  affaire
29  affaire
32  affaire
34  affaire
14  adresse


In [7]:
sklearn.metrics.accuracy_score(y_test,y_predict)

#calcule de la prédiction 

1.0

Première prédiction de mot ! Les mots prédis par le classifieur après entrainement sont en accord avec les données de test à trouver ! 


# Arbre de décision 

In [8]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [9]:
y_predict=clf.predict(X_test)
print(y_predict)
print(y_test)

['adresse' 'affaire' 'adresse' 'affaire' 'affaire' 'affaire' 'affaire'
 'adresse']
      class
6   adresse
30  affaire
9   adresse
26  affaire
29  affaire
32  affaire
34  affaire
14  adresse


In [10]:
sklearn.metrics.accuracy_score(y_test,y_predict)

1.0